# S4S Test Data Generator

This program generate test data


## Configuration values

Configure the values below for 

- Input data directory

In [ ]:
##############################################################
# Configuration values
##############################################################

import os

# Data Folder and File Names
DATA_SRC_DIR                = './data_source'
DATA_OUT_DIR                = './data_output'


# the original inventory data file
INPUT_ORIG_INV_FILE         = os.path.join(DATA_SRC_DIR, 'inventory_parts.csv')
# the sku/item id to convert original inventory file to
INPUT_PART_NUM_LIST_FILE    = os.path.join(DATA_SRC_DIR, '3m_products.xlsx')
# the locations to convert original inventory file to 
INPUT_LOCATION_LIST_FILE    = os.path.join(DATA_SRC_DIR, '3m_plant_list.txt')

# the in-process data file (for debugging and analysis purpose)
TEST_DATA_PROCESS_FILE      = os.path.join(DATA_OUT_DIR, 'data_process.xlsx')
# the test data output file
TEST_DATA_OUTPUT_FILE       = os.path.join(DATA_OUT_DIR, 'testdata_inventory_load.csv')

# Chage the original inventory data columns to below. 
# The process depends on correct identification of part_num, location, and quantity columns
INPUT_ORIG_INV_COLUMN       = ['ignore_1','part_num','location','quantity','ignore_2']

# Change the input_part_num_list_file colmns to below.
# The process depends on correct identification of part_num column
INPUT_PART_NUM_COLUMN       = ['ignore-index', 'ignore-Approval Number', 'ignore-Manufacturer’s Donning Procedure User Instructions', 'ignore-Model Number/ Product Line', 'part_num']

SUPPLIER_SCBN_ID            = '3M_COMPANY_SD'

TENANT_SCBN_ID              = 'S4S_STATE_CA'


In [ ]:
import numpy as np
import pandas as pd
import random


## Create test data file

1. load original inventory file
1. create mapping from original part # to test data's part #
1. create mapping from original location to test data's location

In [ ]:

def read_input_data_file(filepath: str, columns: [str]) -> (pd.DataFrame, [str], [str]):
    # load SCBN supplier list

    input_data_file = pd.read_csv(filepath)

    input_data_file.columns = columns

    part_num_list = input_data_file['part_num'].unique().tolist()

    location_list = input_data_file['location'].unique().tolist()

    return input_data_file, part_num_list, location_list

def read_input_part_num_list(filepath: str, columns:[str]) -> [str]:

    product_list_file = pd.read_excel(filepath,
            0, # Read the first worksheet
            0, # Header is on row 0 (row 1 in Excel's 1base count)
            )
    
    product_list_file.columns = columns

    new_sku_list = product_list_file['part_num'].unique().tolist()

    return new_sku_list


def read_input_location_list(filepath: str, columns:[str]) -> [str]:


    new_locations = pd.read_csv(filepath, header=None)

    new_locations.columns = columns

    new_location_list = new_locations['location'].tolist()

    return new_location_list


orig_inv_file, orig_part_num_list, orig_location_list = read_input_data_file(INPUT_ORIG_INV_FILE, INPUT_ORIG_INV_COLUMN)

# part_num_list_shuffled = part_num_list.copy()
# random.shuffle(part_num_list_shuffled)

new_part_num_list = read_input_part_num_list(INPUT_PART_NUM_LIST_FILE, INPUT_PART_NUM_COLUMN)

new_location_list = read_input_location_list(INPUT_LOCATION_LIST_FILE, ['location'])



## Create maps

In [ ]:

def create_part_num_map(orig_part_num_list:[str], new_part_num_list:[str] ) -> pd.DataFrame:

    orig_part_num_table = pd.DataFrame(orig_part_num_list, columns=['orig_part_num'])

    new_part_num_table = pd.DataFrame(new_part_num_list * int(len(orig_part_num_list)/len(new_part_num_list)+1),
        columns=['new_part_num'])

    part_num_map = orig_part_num_table.join(new_part_num_table)

    return part_num_map


def create_location_map(orig_location_list:[str], new_location_list:[str]) -> pd.DataFrame:

    old_location_table = pd.DataFrame(orig_location_list, columns=['orig_location'])

    new_location_table = pd.DataFrame(new_location_list * int(len(old_location_table)/len(new_location_list)+1), columns=['new_location'])

    location_map = old_location_table.join(new_location_table)

    return location_map



part_num_map = create_part_num_map(orig_part_num_list, new_part_num_list)

location_map = create_location_map(orig_location_list, new_location_list)


## Join the original inventory table to the mapping tables

In [ ]:
inv_new_part_num = orig_inv_file.merge(
    part_num_map,
    how='left',
    left_on = 'part_num',
    right_on = 'orig_part_num',
)

inv_new_partnum_new_location = inv_new_part_num.merge(
    location_map,
    how = 'left',
    left_on = 'location',
    right_on = 'orig_location',
)

test_data_table = inv_new_partnum_new_location.groupby(['new_location', 'new_part_num']).agg({'quantity': ['sum']}).reset_index()

test_data_table.columns = ['location id', 'item id', 'quantity']

test_data_table['uom'] = '3MPAK'

test_data_table['date available from'] = '2020-04-01'

test_data_table['supplier\'s SCBN id'] = SUPPLIER_SCBN_ID

test_data_table['customer\'s SCBN id'] = TENANT_SCBN_ID


## Write the output file and debug file

In [ ]:
if not os.path.exists(DATA_OUT_DIR):
    os.mkdir(DATA_OUT_DIR)

# Save test data as a CSV
test_data_table.to_csv(TEST_DATA_OUTPUT_FILE, index=False)

# persist cdc approved supplier list and ibm supplier list to an Excel

output_writer = pd.ExcelWriter(TEST_DATA_PROCESS_FILE)

location_map.to_excel(output_writer, sheet_name='location map')
part_num_map.to_excel(output_writer, sheet_name='part num map')
# inv_new_partnum_new_location.to_excel(output_writer, sheet_name='test data merged')

output_writer.save()
output_writer.close()

